# 0. Import & Raw 데이터 불러오기

In [1]:
import numpy as np
import pandas as pd
import os

import time
from datetime import timedelta
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 최초 raw 로그 데이터 불러오기
raw = pd.read_csv(os.path.join(os.getcwd(),'log_211201_220301.csv'))

In [3]:
raw.shape

(5948704, 7)

In [4]:
raw.head()

,datetime,user_id,sex,age,page,url,device
0,2022-01-05 00:00:01,U11007,F,36,NHN/한국/네이버/모바일_웹/모바일메인/쇼핑판E/미시판(구)/웹,https://m.naver.com/,Mozilla/5.0 (iPhone; CPU iPhone OS 14_8_1 like...
1,2022-01-05 00:00:02,U14212,M,44,NHN/한국/네이버/모바일_웹/모바일_쇼핑/모바일_스마트스토어/상품상세/상품상세,https://m.smartstore.naver.com/campingrecipe/p...,Mozilla/5.0 (Linux; Android 11; SM-G970N Build...
2,2022-01-05 00:00:03,U14108,M,44,NHN/한국/네이버/모바일_웹/모바일_쇼핑/쇼핑윈도/장보기/브랜드장보기/상품상세/상품상세,https://m.shopping.naver.com/market/necessity/...,Mozilla/5.0 (Linux; Android 11; SM-N986N Build...
3,2022-01-05 00:00:04,U10378,F,45,NHN/한국/네이버/모바일_웹/모바일_쇼핑/쇼핑윈도/아울렛윈도/아울렛윈도(NEW)/...,https://m.shopping.naver.com/outlet/event/2000...,Mozilla/5.0 (Linux; Android 11; SAMSUNG SM-N97...
4,2022-01-05 00:00:05,U13602,F,47,NHN/한국/네이버/모바일_웹/모바일_쇼핑/모바일_스마트스토어/상품상세/상품상세,https://m.smartstore.naver.com/wisemonster/pro...,Mozilla/5.0 (Linux; Android 11; SAMSUNG SM-N98...


# 1. 기초 전처리

## 1.1. 라벨 데이터 merge

In [5]:
# uxlab 2차 레이블 불러오기
labels = pd.read_csv(os.path.join(os.getcwd(),'uxlab_label2.csv'))

In [6]:
labels.head()

,PAGE,Labeling,Num,URL
0,"['모바일_쇼핑', 'Toptop(탑탑)', '리뷰', '원부_포토,동영상_상세n목록']",1,482,https://toptop.naver.com/main/shopping-reviews...
1,"['모바일_쇼핑', '쇼핑윈도', '장보기', '배송지설정', '배송지목록']",1,326,https://m.shopping.naver.com/shipping/list
2,"['모바일_쇼핑', '원쁠딜', '서비스소개']",1,84,https://plusdeal.naver.com/sharepoint
3,"['모바일_쇼핑', '쇼핑윈도', '장보기', '배송지설정', '배송지추가']",1,49,https://m.shopping.naver.com/shipping/form
4,"['모바일_쇼핑', '쇼핑윈도', '펫윈도', '펫등록']",1,22,https://m.swindow.naver.com/pet/profile?petId=...


In [7]:
# datetime 칼럼 자료형 변환
# page 칼럼에서 필요한 부분만 추출해서 PAGE라는 칼럼에 저장(labels와 칼럼 이름 맞춤)
raw['datetime'] = pd.to_datetime(raw['datetime'])
raw['PAGE'] = raw['page'].apply(lambda x: str(x.split('/')[4:]))

In [8]:
# UXLab 라벨 데이터와 raw 데이터 병합
raw = raw.merge(labels,on='PAGE',how='left')

## 1.2. datetime, page 자료형 변환

In [9]:
# datetime 칼럼 자료형 변환 (한번 더)
# 안 쓸 칼럼 (page, Num) drop
raw['datetime'] = pd.to_datetime(raw['datetime'])
raw.drop(['page','Num','URL'],axis=1,inplace=True)

In [10]:
# PAGE 칼럼을 page로 이름 변경
# user_id, datetime 오름차순으로 정렬
raw.rename(columns={'PAGE':'page'},inplace=True)
raw.sort_values(['user_id','datetime'],inplace=True)

## 1.3. Outlier 로그 제거 & 세션 나누기

In [11]:
# 라벨이 *인 데이터들 제거
raw = raw[raw['Labeling']!='*'].reset_index().drop('index',axis=1)

In [12]:
# 숫자인 라벨들만 남았으므로, Labeling 칼럼의 자료형을 int로 변환
raw['Labeling'] = raw['Labeling'].astype(int)

In [13]:
# 60분 이상 차이나면 세션 나눔
# 각 user_id와 세션번호를 조합하여 id 칼럼 생성
raw['session'] = raw.groupby('user_id')['datetime'].apply(lambda x: x.diff().gt('60Min').cumsum())
raw['id'] = raw['user_id']+'_'+raw['session'].astype(str)

In [14]:
# page_stay_time(해당 로그 내에 얼마나 오래 체류했는지) 칼럼 생성
raw['page_stay_time'] = raw.groupby('id')['datetime'].diff()

In [15]:
# 이전 로그와 중복된 URL을 갖는 로그 제거
raw['prev_url'] = raw.groupby('id')['url'].shift(1)
raw = raw[raw['url']!=raw['prev_url']].drop('prev_url',axis=1)

In [16]:
raw.shape

(5394352, 11)

## 1.4. 세션 관련 기본 feature 생성

In [17]:
# session_length (세션 길이) 칼럼 생성
# session_nth (세션 내 몇번째 로그인지) 칼럼 생성
raw = raw.merge(pd.DataFrame(raw.groupby('id')['session'].count()).rename(columns={'session':'session_length'}),on='id',how='left')
raw['session_nth'] = raw.groupby('id').cumcount()+1

In [18]:
# prev_Labeling 칼럼 생성 (해당 세션 내 이전 라벨)
# prev_page 칼럼 생성 (해당 세션 내 이전 페이지)
raw['prev_Labeling'] = raw.groupby('id')['Labeling'].shift(1)
raw['prev_page'] = raw.groupby('id')['page'].shift(1)

In [19]:
# begin_time 칼럼 생성 (해당 세션의 시작 시간)
# end_time 칼럼 생성 (해당 세션의 마지막 시간)
raw = raw.merge(pd.DataFrame(raw.groupby('id')['datetime'].min()).rename(columns={'datetime':'begin_time'}),on='id',how='left')
raw = raw.merge(pd.DataFrame(raw.groupby('id')['datetime'].max()).rename(columns={'datetime':'end_time'}),on='id',how='left')

# 2. Cart, Like, Purchase 데이터와 merge

In [20]:
# 최초 cart, like, purchase 데이터 불러오기
cart = pd.read_csv(os.path.join(os.getcwd(),'cart_20220407.csv'))
like = pd.read_csv(os.path.join(os.getcwd(),'like_20220412.csv'))
purchase = pd.read_csv(os.path.join(os.getcwd(),'purchase_20220407.csv'))

/tmp/ipykernel_5209/2568645029.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  purchase = pd.read_csv(os.path.join(os.getcwd(),'purchase_20220407.csv'))


In [21]:
cart.shape

(212770, 8)

In [22]:
like.shape

(92147, 9)

In [23]:
purchase.shape

(123985, 11)

In [24]:
# 병합을 위해 end_time2 (세션의 마지막 시간에 60분을 더한) 칼럼 생성
raw['end_time2'] = raw['end_time'] + pd.to_timedelta(1, unit='h')

## 2.1. Cart 데이터와 merge

> cart.csv에서 특정 고객의 action이 발생한 경우, 해당 고객의 세션들 중 해당 시간을 포함하는 세션이 있는지 확인



> 포함되는 세션이 있으면, 해당 세션 내에서 시간 상 가장 가까운 로그에 매칭

In [25]:
cart.head()

,user_id,prod_no,prod_nm,event_type,event_ymdt,catg_nm_l,catg_nm_m,catg_nm_s
0,U13824,5998072702,대리석 잼스톤 호일필름 10종세트+투명스톤100개입 클리어수제젤네일아트셀프 명품재료팁레진,com.naver.ncp.cart.product.message.CartProduct...,2021-12-09 08:18:36.764,화장품/미용,네일케어,네일아트
1,U13824,6044012188,뷰젤 기초젤 10ml 17종 [스크래치] 탑 베이스 클리어 오빠 데빌 퍼펙트 멀티 ...,com.naver.ncp.cart.product.message.CartProduct...,2021-12-09 08:21:02.761,화장품/미용,네일케어,매니큐어
2,U13871,5163579376,여성 하이웨스트 융털 세미와이드 밴딩 슬랙스 기모바지,com.naver.ncp.cart.product.message.CartProduct...,2021-12-09 08:28:29.678,패션의류,여성의류,바지
3,U14260,5993477739,포피 플레이타임 인형 허기워기 게임 굿즈 키시미시 파피 뽀삐 Poppy Playtime,com.naver.ncp.cart.product.message.CartProduct...,2021-12-09 08:13:23.755,출산/육아,인형,봉제인형
4,U13824,2540097319,뷰젤 쉬폰 무광 매트 탑젤 10g 논와이프 탑 네일아트재료,com.naver.ncp.cart.product.message.CartProduct...,2021-12-09 08:16:08.172,화장품/미용,네일케어,매니큐어


In [26]:
# cart 데이터를 user_id, 시간 순으로 정렬
# 병합을 위해, cart 데이터의 시간을 초 단위까지만 표시
# cart 데이터는 UTC+9시간으로 되어 있어, raw와 시간대 맞추기 위해 9시간 빼줌
# cart의 event_type 칼럼에서 필요한 부분만 추출
cart.sort_values(by=['user_id','event_ymdt'],inplace=True)
cart['event_ymdt'] = pd.to_datetime(cart['event_ymdt']).round('s')
cart['event_ymdt'] = cart['event_ymdt'] - timedelta(hours=9)   
cart['event_type'] = cart['event_type'].apply(lambda x: x.split('.')[-1])
cart.head()

,user_id,prod_no,prod_nm,event_type,event_ymdt,catg_nm_l,catg_nm_m,catg_nm_s
185318,U10001,5633129636,묘야 레인보우 아크릴스탠드,CartProductCreated,2021-12-05 19:38:32,생활/건강,수집품,모형/프라모델/피규어
185316,U10001,6068019895,(예약) 묘야 2022 탁상달력,CartProductCreated,2021-12-05 19:39:04,생활/건강,문구/사무용품,다이어리/플래너
185317,U10001,5633129636,묘야 레인보우 아크릴스탠드,CartProductDeleted,2021-12-05 19:39:25,생활/건강,수집품,모형/프라모델/피규어
185315,U10001,6068019895,(예약) 묘야 2022 탁상달력,CartProductDeleted,2021-12-05 19:39:46,생활/건강,문구/사무용품,다이어리/플래너
166978,U10001,2341342392,휴지걸이 욕실 화장실 스텐 접착식,CartProductCreated,2021-12-12 16:39:29,생활/건강,욕실용품,욕실용기/홀더


In [27]:
# 세션의 첫 시간부터 마지막 시간+60분 사이에 cart에 해당 고객의 action이 있으면 merge

# raw 내 모든 데이터의 datetime과 cart 내 모든 데이터의 datetime을 비교하는 것은 비효율적
# 날짜가 같은 데이터끼리만 비교하는 것이 효율적이므로, raw와 cart에 각각 date 칼럼 생성
# but 이렇게 되면, 세션의 마지막 시간이 23시~00시 사이이고, cart의 action이 00시~01시 사이에 발생한 경우 merge가 누락됨.
# 그래서 cart에 date2 (cart의 datetime에서 하루를 뺀 것의 날짜) 칼럼 생성
raw['date'] = raw['datetime'].dt.floor('d')
cart['date'] = cart['event_ymdt'].dt.floor('d')
cart['date2'] = cart['event_ymdt'].dt.floor('d') - timedelta(days=1)

In [28]:
# raw의 date와 cart의 date가 일치하는 데이터끼리 merge
# 세션의 첫 시간부터 마지막 시간+60분 사이에 cart에 해당 고객의 action이 있는 것만 남김
log_cart = raw.merge(cart,on=['user_id','date'],how='inner')
log_cart = log_cart[((log_cart['event_ymdt'] - log_cart['begin_time'])>timedelta(0))*((log_cart['end_time2']-log_cart['event_ymdt'])>timedelta(0))==1]
log_cart.sort_values(by=['id','datetime'],inplace=True)

/home/jipal28/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:214: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead.
  warnings.warn(


In [29]:
# raw의 date와 cart의 date2 (cart의 date에서 1일 뺀 것)가 일치하는 데이터끼리 merge
# 세션의 첫 시간부터 마지막 시간+60분 사이에 cart에 해당 고객의 action이 있는 것만 남김
log_cart2 = raw.merge(cart,left_on=['user_id','date'],right_on=['user_id','date2'],how='inner')
log_cart2 = log_cart2[((log_cart2['event_ymdt'] - log_cart2['begin_time'])>timedelta(0))*((log_cart2['end_time2']-log_cart2['event_ymdt'])>timedelta(0))==1]
log_cart2.sort_values(by=['id','datetime'],inplace=True)

/home/jipal28/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:214: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead.
  warnings.warn(


In [30]:
# 위 두 merge 데이터 합침
log_cart = pd.concat([log_cart,log_cart2],axis=0)

# 각 세션별로, cart action과 시간 상 가장 가까운 로그에 매칭시킴
log_cart['diff'] = ((log_cart['event_ymdt'] - log_cart['datetime']).astype('timedelta64[s]'))**2
log_cart = log_cart.merge(pd.DataFrame(log_cart.groupby('id')['diff'].min()).rename(columns={'diff':'min_diff'}),on='id',how='left')
log_cart = log_cart[(log_cart['diff']-log_cart['min_diff'])**2<0.1]
log_cart.drop_duplicates(subset=['id','event_ymdt'],keep='last',inplace=True)
log_cart.drop('min_diff',axis=1,inplace=True)
log_cart['cart_id'] = 1

In [31]:
# 전체 cart 데이터 중 약 30% 정도만 매칭되는 로그 존재 
# (나머지는 웹 상에서의 action으로 추정)
log_cart.shape[0] / cart.shape[0]

0.2997509047328101

In [32]:
# 사용하지 않을 칼럼들 드랍
# 이후 전체 로그 데이터와 병합될 때 혼동을 피하기 위해 칼럼 이름들 앞에 'cart' 붙임
log_cart.drop(['cart_id','end_time2','date','date_x','date_y','date2'],axis=1,inplace=True)
log_cart.rename(columns={'prod_no':'cart_prod_no','prod_nm':'cart_prod_nm','event_type':'cart_event_type','event_ymdt':'cart_event_ymdt',
                                    'catg_nm_l':'cart_catg_nm_l','catg_nm_m':'cart_catg_nm_m','catg_nm_s':'cart_catg_nm_s','diff':'log_cart_time_diff'},inplace=True)

In [33]:
log_cart.columns

Index(['datetime', 'user_id', 'sex', 'age', 'url', 'device', 'page',
       'Labeling', 'session', 'id', 'page_stay_time', 'session_length',
       'session_nth', 'prev_Labeling', 'prev_page', 'begin_time', 'end_time',
       'cart_prod_no', 'cart_prod_nm', 'cart_event_type', 'cart_event_ymdt',
       'cart_catg_nm_l', 'cart_catg_nm_m', 'cart_catg_nm_s',
       'log_cart_time_diff'],
      dtype='object')

## 2.2. Like 데이터와 merge

> like.csv에서 특정 고객의 action이 발생한 경우, 해당 고객의 세션들 중 해당 시간을 포함하는 세션이 있는지 확인



> 포함되는 세션이 있으면, 해당 세션 내에서 시간 상 가장 가까운 로그에 매칭

In [34]:
like.head()

,user_id,keep_stat_cd,prod_no,prod_nm,keep_reg_ymdt,catg_nm_l,catg_nm_m,catg_nm_s,dt
0,U14965,D,5171148345,"아에르 보건용마스크 KF80 대형,화이트(2매입) 100매(무료배송)",2021-12-31 10:08,생활/건강,건강관리용품,먼지차단마스크,20211231
1,U11569,A,4961583997,드로잉 패브릭 포스터 1:1 맞춤 제작 커스텀 인테리어 소품,2022-02-15 17:07,가구/인테리어,인테리어소품,액자,20220215
2,U10919,D,5277402754,아이폰 맥세이프 카드지갑 서드파티 스트래퍼스 소가죽 카드홀더 애플 케이스,2021-12-30 9:51,디지털/가전,휴대폰액세서리,기타휴대폰액세서리,20211230
3,U14377,A,3730339623,와이젠 돈풍기 원적외선 튜브히터 곱창난로 등유 주름관히터 팬형/90평형,2021-12-26 18:34,디지털/가전,계절가전,온풍기,20211226
4,U10369,A,6121954606,인조가죽 라인 퍼자켓 (AZW1QJ02A)/에이비에프지,2022-01-31 18:46,패션의류,여성의류,재킷,20220131


In [35]:
# like 데이터를 user_id, 시간 순으로 정렬
# like 데이터는 UTC로 되어 있어, raw와 시간대 맞추기 위한 조작 필요x.
# 사용하지 않을 칼럼 (dt) drop
like.sort_values(by=['user_id','keep_reg_ymdt'],inplace=True)
like['keep_reg_ymdt'] = pd.to_datetime(like['keep_reg_ymdt'])
like.drop('dt',axis=1,inplace=True)
like.head()

,user_id,keep_stat_cd,prod_no,prod_nm,keep_reg_ymdt,catg_nm_l,catg_nm_m,catg_nm_s
81182,U10001,D,4698548361,칸지샵 초음파가습기 생수병가습기 사무실가습기,2021-12-03 11:34:00,디지털/가전,계절가전,가습기
63931,U10001,D,2625074850,[해리슨] 댑맨투맨 MJB1354,2021-12-04 04:26:00,NaN,NaN,NaN
32369,U10001,A,5749682559,캉골 코트 메신저백 2046 카키,2021-12-07 04:03:00,패션잡화,남성가방,크로스백
72497,U10001,D,5140526501,[해리슨] 나그랑 배색 반지퍼 맨투맨 MJB1336,2021-12-08 08:46:00,패션의류,남성의류,티셔츠
10185,U10001,A,3584920847,신개념 문풍지 외풍차단 창문 문틈 창틀 바람막이 틈새막이 틈막이,2021-12-27 02:13:00,생활/건강,생활용품,생활잡화


In [36]:
# 세션의 첫 시간부터 마지막 시간+60분 사이에 like에 해당 고객의 action이 있으면 merge

# raw 내 모든 데이터의 datetime과 like 내 모든 데이터의 datetime을 비교하는 것은 비효율적
# 날짜가 같은 데이터끼리만 비교하는 것이 효율적이므로, raw와 like에 각각 date 칼럼 생성
# but 이렇게 되면, 세션의 마지막 시간이 23시~00시 사이이고, like의 action이 00시~01시 사이에 발생한 경우 merge가 누락됨.
# 그래서 like에 date2 (like의 datetime에서 하루를 뺀 것의 날짜) 칼럼 생성
raw['date'] = raw['datetime'].dt.floor('d')
like['date'] = like['keep_reg_ymdt'].dt.floor('d')
like['date2'] = like['keep_reg_ymdt'].dt.floor('d') - timedelta(days=1)

In [37]:
# raw의 date와 like의 date가 일치하는 데이터끼리 merge
# 세션의 첫 시간부터 마지막 시간+60분 사이에 like에 해당 고객의 action이 있는 것만 남김
log_like = raw.merge(like,on=['user_id','date'],how='inner')
log_like = log_like[((log_like['keep_reg_ymdt'] - log_like['begin_time'])>timedelta(0))*((log_like['end_time2']-log_like['keep_reg_ymdt'])>timedelta(0))==1]
log_like.sort_values(by=['id','datetime'],inplace=True)

/home/jipal28/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:214: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead.
  warnings.warn(


In [38]:
# raw의 date와 like의 date2 (like의 date에서 1일 뺀 것)가 일치하는 데이터끼리 merge
# 세션의 첫 시간부터 마지막 시간+60분 사이에 like에 해당 고객의 action이 있는 것만 남김
log_like2 = raw.merge(like,left_on=['user_id','date'],right_on=['user_id','date2'],how='inner')
log_like2 = log_like2[((log_like2['keep_reg_ymdt'] - log_like2['begin_time'])>timedelta(0))*((log_like2['end_time2']-log_like2['keep_reg_ymdt'])>timedelta(0))==1]
log_like2.sort_values(by=['id','datetime'],inplace=True)

/home/jipal28/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:214: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead.
  warnings.warn(


In [39]:
# 위 두 merge 데이터 합침
log_like = pd.concat([log_like,log_like2],axis=0)

# 각 세션별로,like action과 시간 상 가장 가까운 로그에 매칭시킴
log_like['diff'] = ((log_like['keep_reg_ymdt'] - log_like['datetime']).astype('timedelta64[s]'))**2
log_like = log_like.merge(pd.DataFrame(log_like.groupby('id')['diff'].min()).rename(columns={'diff':'min_diff'}),on='id',how='left')
log_like = log_like[(log_like['diff']-log_like['min_diff'])**2<0.1]
log_like.drop_duplicates(subset=['id','keep_reg_ymdt'],keep='last',inplace=True)
log_like.drop('min_diff',axis=1,inplace=True)
log_like['like_id'] = 1

In [40]:
# 전체 like 데이터 중 약 24.7% 정도만 매칭되는 로그 존재 
# (나머지는 웹 상에서의 action으로 추정)
log_like.shape[0] / like.shape[0]

0.2465842621029442

In [41]:
# 사용하지 않을 칼럼들 드랍
# 이후 전체 로그 데이터와 병합될 때 혼동을 피하기 위해 칼럼 이름들 앞에 'like' 붙임
log_like.drop(['like_id','end_time2','date','date2','date_x','date_y'],axis=1,inplace=True)
log_like.rename(columns={'keep_stat_cd':'like_keep_stat_cd','prod_no':'like_prod_no','prod_nm':'like_prod_nm','keep_reg_ymdt':'like_keep_reg_ymdt',
                                    'catg_nm_l':'like_catg_nm_l','catg_nm_m':'like_catg_nm_m','catg_nm_s':'like_catg_nm_s','diff':'log_like_time_diff'},inplace=True)

In [42]:
log_like.columns

Index(['datetime', 'user_id', 'sex', 'age', 'url', 'device', 'page',
       'Labeling', 'session', 'id', 'page_stay_time', 'session_length',
       'session_nth', 'prev_Labeling', 'prev_page', 'begin_time', 'end_time',
       'like_keep_stat_cd', 'like_prod_no', 'like_prod_nm',
       'like_keep_reg_ymdt', 'like_catg_nm_l', 'like_catg_nm_m',
       'like_catg_nm_s', 'log_like_time_diff'],
      dtype='object')

## 2.3. Purchase 데이터와 merge

> purchase.csv에서 특정 고객의 action이 발생한 경우, 해당 고객의 세션들 중 해당 시간을 포함하는 세션이 있는지 확인



> 포함되는 세션이 있으면, 해당 세션 내에서 시간 상 가장 가까운 로그에 매칭

In [43]:
purchase.head()

,user_id,prod_order_no,order_no,order_ymdt,prod_no,prod_nm,gmv,flag,catg_nm_l,catg_nm_m,catg_nm_s
0,U13726,2021121350387480,2021121384586370,21-12-13 16:08,5776267167,(정기 구독) 삼다수 그린(무라벨) 2.0L 12개입,11800,주문,식품,음료,생수
1,U11109,2021121357168770,2021121388670100,21-12-13 19:29,442645012,1+1 나투젠 아쿠아 수용성 러브 바디 글라이드 마사지 젤 450ml / 최신유통기한,1990,주문,화장품/미용,바디케어,바디슬리밍
2,U12073,2021121358786900,2021121389670600,21-12-13 20:16,5819493126,콜핑 남성 겨울 본딩 기모 등산바지 코나드(CL)(남) KSP9550M,36400,주문,스포츠/레저,등산,등산의류
3,U11994,2021121411367550,2021121424919800,21-12-14 21:40,4823519851,국산 2021 베이직 멜란지 골지 여자양말 무지 중장목 컬러 가을 학생 흰 패션양말...,1650,주문,패션잡화,양말,여성양말
4,U13461,2021121411963900,2021121425276270,21-12-14 21:56,4329911012,여성 국산 면레깅스 스판레깅스 7부 8부 9부 기모레깅스 겨울융털타이즈 기본아이템,5200,주문,패션의류,여성의류,레깅스


In [44]:
# purchase 데이터를 user_id, 시간 순으로 정렬
# purchase 데이터는 UTC로 되어 있어, raw와 시간대 맞추기 위한 조작 필요x
# purchase 데이터의 datetime이 '21년 이런 식으로 되어 있어, 앞에 '20' 붙여줌
purchase.sort_values(by=['user_id','order_ymdt'],inplace=True)
purchase['order_ymdt'] = pd.to_datetime(purchase['order_ymdt'].apply(lambda x: '20'+x))
purchase.head()

,user_id,prod_order_no,order_no,order_ymdt,prod_no,prod_nm,gmv,flag,catg_nm_l,catg_nm_m,catg_nm_s
18189,U10001,2021121299854660,2021121260930360,2021-12-12 16:40:00,2341342392,휴지걸이 욕실 화장실 스텐 접착식,6800,주문,생활/건강,욕실용품,욕실용기/홀더
27586,U10001,2021121299854650,2021121260930360,2021-12-12 16:40:00,2341342392,휴지걸이 욕실 화장실 스텐 접착식,4300,주문,생활/건강,욕실용품,욕실용기/홀더
103086,U10001,2021121299854670,2021121260930360,2021-12-12 16:40:00,2341342392,휴지걸이 욕실 화장실 스텐 접착식,2500,주문,생활/건강,욕실용품,욕실용기/홀더
35219,U10001,2021121210357650,2021121261235960,2021-12-12 16:57:00,5128769981,비에스에스 기프트 국산 식약처 의약외품 KFAD 비말차단 일회용 덴탈 마스크 50매,39600,주문,생활/건강,건강관리용품,먼지차단마스크
13293,U10001,2021122963744780,2021122956598510,2021-12-29 18:07:00,6089361331,타이틀리스트 골프공 Pro v1 프로 v1x 홀인원 기념품 선물 행사 용품 인쇄 제...,38000,주문,스포츠/레저,골프,골프공


In [45]:
# 세션의 첫 시간부터 마지막 시간+60분 사이에 purchase에 해당 고객의 action이 있으면 merge

# raw 내 모든 데이터의 datetime과 purchase 내 모든 데이터의 datetime을 비교하는 것은 비효율적
# 날짜가 같은 데이터끼리만 비교하는 것이 효율적이므로, raw와 purchase에 각각 date 칼럼 생성
# but 이렇게 되면, 세션의 마지막 시간이 23시~00시 사이이고, purchase의 action이 00시~01시 사이에 발생한 경우 merge가 누락됨.
# 그래서 purchase에 date2 (purchase의 datetime에서 하루를 뺀 것의 날짜) 칼럼 생성
raw['date'] = raw['datetime'].dt.floor('d')
purchase['date'] = purchase['order_ymdt'].dt.floor('d')
purchase['date2'] = purchase['order_ymdt'].dt.floor('d') - timedelta(days=1)

In [46]:
# raw의 date와 purchase의 date가 일치하는 데이터끼리 merge
# 세션의 첫 시간부터 마지막 시간+60분 사이에 purchase에 해당 고객의 action이 있는 것만 남김
log_purchase = raw.merge(purchase,on=['user_id','date'],how='inner')
log_purchase = log_purchase[((log_purchase['order_ymdt'] - log_purchase['begin_time'])>timedelta(0))*((log_purchase['end_time2']-log_purchase['order_ymdt'])>timedelta(0))==1]
log_purchase.sort_values(by=['id','datetime'],inplace=True)

/home/jipal28/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:214: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead.
  warnings.warn(


In [47]:
# raw의 date와 purchase의 date2 (purchase의 date에서 1일 뺀 것)가 일치하는 데이터끼리 merge
# 세션의 첫 시간부터 마지막 시간+60분 사이에 purchase에 해당 고객의 action이 있는 것만 남김
log_purchase2 = raw.merge(purchase,left_on=['user_id','date'],right_on=['user_id','date2'],how='inner')
log_purchase2 = log_purchase2[((log_purchase2['order_ymdt'] - log_purchase2['begin_time'])>timedelta(0))*((log_purchase2['end_time2']-log_purchase2['order_ymdt'])>timedelta(0))==1]
log_purchase2.sort_values(by=['id','datetime'],inplace=True)

/home/jipal28/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:214: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead.
  warnings.warn(


In [48]:
# 위 두 merge 데이터 합침
log_purchase = pd.concat([log_purchase,log_purchase2],axis=0)

# 각 세션별로,purchase action과 시간 상 가장 가까운 로그에 매칭시킴
log_purchase['diff'] = ((log_purchase['order_ymdt'] - log_purchase['datetime']).astype('timedelta64[s]'))**2
log_purchase = log_purchase.merge(pd.DataFrame(log_purchase.groupby('id')['diff'].min()).rename(columns={'diff':'min_diff'}),on='id',how='left')
log_purchase = log_purchase[(log_purchase['diff']-log_purchase['min_diff'])**2<0.1]
log_purchase.drop_duplicates(subset=['id','order_ymdt'],keep='last',inplace=True)
log_purchase.drop('min_diff',axis=1,inplace=True)
log_purchase['purchase_id'] = 1

In [49]:
# 전체 purchase 데이터 중 약 34.6% 정도만 매칭되는 로그 존재 
# (나머지는 웹 상에서의 action으로 추정)
log_purchase.shape[0] / purchase.shape[0]

0.34646126547566236

In [50]:
# 사용하지 않을 칼럼들 드랍
# 이후 전체 로그 데이터와 병합될 때 혼동을 피하기 위해 칼럼 이름들 앞에 'purchase' 붙임
log_purchase.drop(['purchase_id','end_time2','date','date2','date_x','date_y'],axis=1,inplace=True)
log_purchase.rename(columns={'prod_order_no':'purchase_prod_order_no','order_no':'purchase_order_no','order_ymdt':'purchase_order_ymdt',
                                            'prod_no':'purchase_prod_no','prod_nm':'purchase_prod_nm','gmv':'purchase_gmv','flag':'purchase_flag',
                                            'catg_nm_l':'purchase_catg_nm_l','catg_nm_m':'purchase_catg_nm_m','catg_nm_s':'purchase_catg_nm_s','diff':'log_purchase_time_diff'},inplace=True)

In [51]:
log_purchase.columns

Index(['datetime', 'user_id', 'sex', 'age', 'url', 'device', 'page',
       'Labeling', 'session', 'id', 'page_stay_time', 'session_length',
       'session_nth', 'prev_Labeling', 'prev_page', 'begin_time', 'end_time',
       'purchase_prod_order_no', 'purchase_order_no', 'purchase_order_ymdt',
       'purchase_prod_no', 'purchase_prod_nm', 'purchase_gmv', 'purchase_flag',
       'purchase_catg_nm_l', 'purchase_catg_nm_m', 'purchase_catg_nm_s',
       'log_purchase_time_diff'],
      dtype='object')

## 2.4. raw, cart, like, purchase 전체 merge

In [52]:
# raw에 log_cart, log_like, log_purchase 병합
# 병합 시, raw와 공통으로 갖고 있는 칼럼들은 제거하고 merge 
# (id랑 session_nth 기준으로 병합하므로, 이들 칼럼은 공통으로 갖고 있어도 제거x)
raw_all = raw.merge(log_cart.drop(list((set(log_cart.columns).intersection(set(raw.columns))).difference(set(['id','session_nth']))),axis=1),on=['id','session_nth'],how='left')
raw_all = raw_all.merge(log_like.drop(list((set(log_like.columns).intersection(set(raw.columns))).difference(set(['id','session_nth']))),axis=1),on=['id','session_nth'],how='left')
raw_all = raw_all.merge(log_purchase.drop(list((set(log_purchase.columns).intersection(set(raw.columns))).difference(set(['id','session_nth']))),axis=1),on=['id','session_nth'],how='left')
raw_all.head(1)

,datetime,user_id,sex,age,url,device,page,Labeling,session,id,...,purchase_order_no,purchase_order_ymdt,purchase_prod_no,purchase_prod_nm,purchase_gmv,purchase_flag,purchase_catg_nm_l,purchase_catg_nm_m,purchase_catg_nm_s,log_purchase_time_diff
0,2021-12-01 06:31:59,U10001,M,31,https://msearch.shopping.naver.com/search/all?...,Mozilla/5.0 (Linux; Android 11; SM-G998N Build...,"['모바일_쇼핑', '검색_쇼핑검색', '쇼핑검색(도구필터링제외)']",5,0,U10001_0,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# cart_id 칼럼(해당 세션에 cart action이 한번이라도 있으면 1, 없으면 0) 생성
# like_id 칼럼(해당 세션에 like action이 한번이라도 있으면 1, 없으면 0) 생성
# purchase_id 칼럼(해당 세션에 purchase action이 한번이라도 있으면 1, 없으면 0) 생성
raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['cart_event_ymdt'].count().gt(0).astype(float)).rename(columns={'cart_event_ymdt':'cart_id'}),on='id',how='left')
raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['like_keep_reg_ymdt'].count().gt(0).astype(float)).rename(columns={'like_keep_reg_ymdt':'like_id'}),on='id',how='left')
raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['purchase_order_ymdt'].count().gt(0).astype(float)).rename(columns={'purchase_order_ymdt':'purchase_id'}),on='id',how='left')

In [54]:
# raw_all에서 더 이상 안 쓰일 칼럼 (end_time2, date) 제거
raw_all.drop(['end_time2','date'],axis=1,inplace=True)

In [55]:
raw_all.columns

Index(['datetime', 'user_id', 'sex', 'age', 'url', 'device', 'page',
       'Labeling', 'session', 'id', 'page_stay_time', 'session_length',
       'session_nth', 'prev_Labeling', 'prev_page', 'begin_time', 'end_time',
       'cart_prod_no', 'cart_prod_nm', 'cart_event_type', 'cart_event_ymdt',
       'cart_catg_nm_l', 'cart_catg_nm_m', 'cart_catg_nm_s',
       'log_cart_time_diff', 'like_keep_stat_cd', 'like_prod_no',
       'like_prod_nm', 'like_keep_reg_ymdt', 'like_catg_nm_l',
       'like_catg_nm_m', 'like_catg_nm_s', 'log_like_time_diff',
       'purchase_prod_order_no', 'purchase_order_no', 'purchase_order_ymdt',
       'purchase_prod_no', 'purchase_prod_nm', 'purchase_gmv', 'purchase_flag',
       'purchase_catg_nm_l', 'purchase_catg_nm_m', 'purchase_catg_nm_s',
       'log_purchase_time_diff', 'cart_id', 'like_id', 'purchase_id'],
      dtype='object')

# 3. Outlier 세션 제거

In [56]:
# 길이가 2 이하인 세션 제거
raw_all = raw_all[raw_all['session_length']>2]

In [57]:
raw_all.shape

(5003128, 47)

In [58]:
# distinct_page_num(해당 세션 내에 몇 개의 페이지(라벨보다 종류 많음)가 있는지) 칼럼 생성
# distinct_label_num(해당 세션 내에 몇 개의 라벨(페이지보다 종류 적음)가 있는지) 칼럼 생성
raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['page'].nunique()).rename(columns={'page':'distinct_page_num'}),on='id',how='left')
raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['Labeling'].nunique()).rename(columns={'Labeling':'distinct_label_num'}),on='id',how='left')

In [59]:
# 모두 같은 페이지로만 이루어진 세션 제거
raw_all = raw_all[raw_all['distinct_page_num']>1]

In [60]:
raw_all.shape

(4946055, 49)

# 4. 세션별 feature 생성

## 4.1. 로그/세션 내 체류시간 관련

In [61]:
# max_page_stay_time (세션 내 로그 최대 체류 시간) 칼럼 생성
# min_page_stay_time (세션 내 로그 최소 체류 시간) 칼럼 생성
raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['page_stay_time'].max()).rename(columns={'page_stay_time':'max_page_stay_time'}),on='id',how='left')
raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['page_stay_time'].min()).rename(columns={'page_stay_time':'min_page_stay_time'}),on='id',how='left')

In [62]:
# mean_page_stay_time (세션 내 로그 평균 체류 시간) 칼럼 생성
# 그냥 .groupby.mean()하면 평균 계산의 분모가 1씩 줄어든 상태로 계산됨
# 그래서 .groupby.sum()을 .groupby.count()+1 로 나눠서 구함
raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['page_stay_time'].sum() / (raw_all.groupby('id')['page_stay_time'].count()+1)).rename(columns={'page_stay_time':'mean_page_stay_time'}),on='id',how='left')

In [63]:
# session_stay_time (해당 세션 내 총 체류 시간) 칼럼 생성
raw_all['session_stay_time'] = raw_all['end_time'] - raw_all['begin_time']

In [64]:
# 생성된 체류시간 칼럼들의 dtype은 모두 timedelta
# 분석의 편의성을 위해 해당 칼럼들을 모두 초 단위로 float 자료형으로 변환
raw_all['page_stay_time'] = raw_all['page_stay_time'].astype('timedelta64[s]')
raw_all['max_page_stay_time'] = raw_all['max_page_stay_time'].astype('timedelta64[s]')
raw_all['min_page_stay_time'] = raw_all['min_page_stay_time'].astype('timedelta64[s]')
raw_all['mean_page_stay_time'] = raw_all['mean_page_stay_time'].astype('timedelta64[s]')
raw_all['session_stay_time'] = raw_all['session_stay_time'].astype('timedelta64[s]')

## 4.2. 세션 시작으로부터의 시간

In [65]:
# time_from_begin(해당 로그가 세션 시작 시간으로부터 얼마나 경과한 시간에 발생했는지) 칼럼 생성
raw_all['time_from_begin'] = raw_all['datetime'] - raw_all['begin_time']

In [66]:
# purchase_page_reach_time (21번 라벨을 방문한 경우, 해당 세션의 시작으로부터 해당 방문까지 얼마나 시간 걸렸는지) 칼럼 생성
time_from_begin = raw_all.drop_duplicates(subset=['id','Labeling'])[['id','Labeling','time_from_begin']].rename(columns={'time_from_begin':'purchase_page_reach_time'})
raw_all = raw_all.merge(time_from_begin[time_from_begin['Labeling']==21][['id','purchase_page_reach_time']],on='id',how='left')

In [67]:
# 생성된 체류시간 칼럼들의 dtype은 모두 timedelta
# 분석의 편의성을 위해 해당 칼럼들을 모두 초 단위로 float 자료형으로 변환
raw_all['time_from_begin'] = raw_all['time_from_begin'].astype('timedelta64[s]')
raw_all['purchase_page_reach_time'] = raw_all['purchase_page_reach_time'].astype('timedelta64[s]')

## 4.3. Recursion (특정 라벨 반복) 수

In [68]:
# recursion_4 (해당 세션 내 4번 라벨 반복 수) 칼럼 생성
recursion_4 = raw_all.groupby(['id','Labeling'])['prev_Labeling'].value_counts().unstack()[4].unstack()[4]
raw_all = raw_all.merge(pd.DataFrame(recursion_4).rename(columns={4:'recursion_4'}),on='id',how='left')

In [69]:
# recursion_5 (해당 세션 내 5번 라벨 반복 수) 칼럼 생성
recursion_5 = raw_all.groupby(['id','Labeling'])['prev_Labeling'].value_counts().unstack()[5].unstack()[5]
raw_all = raw_all.merge(pd.DataFrame(recursion_5).rename(columns={5:'recursion_5'}),on='id',how='left')

In [70]:
# recursion_6 (해당 세션 내 6번 라벨 반복 수) 칼럼 생성
recursion_6 = raw_all.groupby(['id','Labeling'])['prev_Labeling'].value_counts().unstack()[6].unstack()[6]
raw_all = raw_all.merge(pd.DataFrame(recursion_6).rename(columns={6:'recursion_6'}),on='id',how='left')

In [71]:
# recursion_8 (해당 세션 내 8번 라벨 반복 수) 칼럼 생성
recursion_8 = raw_all.groupby(['id','Labeling'])['prev_Labeling'].value_counts().unstack()[8].unstack()[8]
raw_all = raw_all.merge(pd.DataFrame(recursion_8).rename(columns={8:'recursion_8'}),on='id',how='left')

## 4.4. Cart / Like / Purchase 수

In [72]:
# cart_add_num (해당 세션 내에 cart action의 수) 칼럼 추가
# like_add_num (해당 세션 내에 like action의 수) 칼럼 추가
# purchase_num (해당 세션 내에 purchase action의 수) 칼럼 추가
raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['cart_event_ymdt'].count()).rename(columns={'cart_event_ymdt':'cart_add_num'}),on='id',how='left')
raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['like_keep_reg_ymdt'].count()).rename(columns={'like_keep_reg_ymdt':'like_add_num'}),on='id',how='left')
raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['purchase_order_ymdt'].count()).rename(columns={'purchase_order_ymdt':'purchase_num'}),on='id',how='left')

## 4.5. 세션 내 특정 라벨 관련 feature

In [73]:
# session_begin_label (해당 세션의 첫번째 로그의 라벨) 칼럼 추가
raw_all = raw_all.merge(raw_all.drop_duplicates(subset='id',keep='first')[['id','Labeling']].rename(columns={'Labeling':'session_begin_label'}),on='id',how='left')

In [74]:
# session_end_label (해당 세션의 마지막 로그의 라벨) 칼럼 추가
raw_all = raw_all.merge(raw_all.drop_duplicates(subset='id',keep='last')[['id','Labeling']].rename(columns={'Labeling':'session_end_label'}),on='id',how='left')

In [75]:
# review_page_num (해당 세션에 7번 라벨 로그가 몇 개 있는지) 칼럼 추가
review_page_num = pd.DataFrame(raw_all.groupby('id')['Labeling'].value_counts().unstack()[7]).rename(columns={7:'review_page_num'})
raw_all = raw_all.merge(review_page_num,on='id',how='left')

In [76]:
raw_all.shape

(4946055, 65)

In [77]:
raw_all.columns

Index(['datetime', 'user_id', 'sex', 'age', 'url', 'device', 'page',
       'Labeling', 'session', 'id', 'page_stay_time', 'session_length',
       'session_nth', 'prev_Labeling', 'prev_page', 'begin_time', 'end_time',
       'cart_prod_no', 'cart_prod_nm', 'cart_event_type', 'cart_event_ymdt',
       'cart_catg_nm_l', 'cart_catg_nm_m', 'cart_catg_nm_s',
       'log_cart_time_diff', 'like_keep_stat_cd', 'like_prod_no',
       'like_prod_nm', 'like_keep_reg_ymdt', 'like_catg_nm_l',
       'like_catg_nm_m', 'like_catg_nm_s', 'log_like_time_diff',
       'purchase_prod_order_no', 'purchase_order_no', 'purchase_order_ymdt',
       'purchase_prod_no', 'purchase_prod_nm', 'purchase_gmv', 'purchase_flag',
       'purchase_catg_nm_l', 'purchase_catg_nm_m', 'purchase_catg_nm_s',
       'log_purchase_time_diff', 'cart_id', 'like_id', 'purchase_id',
       'distinct_page_num', 'distinct_label_num', 'max_page_stay_time',
       'min_page_stay_time', 'mean_page_stay_time', 'session_stay_time',
    

# 5. 검색/탐색 유입 구분

## 5.1. 검색유입 구분

In [78]:
# is_query_page(url에 'query'가 들어가 있는지 여부) 칼럼 생성
raw_all['is_query_page'] = raw_all['url'].apply(lambda x: 'query' in x)

In [79]:
# 검색 유입 기준 : 세션 시작 후 30초 이내 & 2번째 로그 이내에 query_page 등장
# under_30sec(세션 시작 후 30초 이내인지) 칼럼 생성
# under_2th(세션 시작 후 2번째 이내 로그인지) 칼럼 생성
# search_in_log(세션 시작 후 30초 이내 & 2번째 로그 이내 & query page) 칼럼 생성
raw_all['under_30sec'] = pd.to_timedelta(raw_all['time_from_begin'],unit='s')<=pd.Timedelta(seconds=30)
raw_all['under_2th'] = raw_all['session_nth']<=2
raw_all['search_in_log'] = raw_all['under_30sec'] * raw_all['under_2th'] * raw_all['is_query_page']

/home/jipal28/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:214: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead.
  warnings.warn(


In [80]:
# search_in(해당 세션이 검색 유입 세션인지) 칼럼 생성
raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['search_in_log'].sum().gt(0)).astype(float).rename(columns={'search_in_log':'search_in'}),on='id',how='left')

In [81]:
# 더 이상 쓰지 않을 칼럼들 drop
raw_all.drop(['under_30sec','under_2th','search_in_log'],axis=1,inplace=True)

In [82]:
raw_all.shape

(4946055, 67)

In [83]:
raw_all.drop_duplicates('id')['search_in'].value_counts()

0.0    252550
1.0     99357
Name: search_in, dtype: int64

## 5.2. 탐색유입 구분

In [84]:
# 탐색 유입 기준 : 검색유입을 제외한 세션 중, 결제/배송 로그만으로 이루어진 세션들을 제외한 나머지 세션
# is_trans_revisit(결제/배송에 해당하는 페이지를 방문한 로그인지 여부) 칼럼 생성
trans_revisit = [15, 21, 22, 23,17, 18, 19, 20, 25, 26]
raw_all['is_trans_revisit'] = raw_all['Labeling'].apply(lambda x: x in trans_revisit)

In [85]:
# trans_revisit_sum(해당 세션 내의 trans_revisit 로그 개수) 칼럼 생성
raw_all = raw_all.merge(pd.DataFrame(raw_all.groupby('id')['is_trans_revisit'].sum()).rename(columns={'is_trans_revisit':'trans_revisit_sum'}),on='id',how='left')

In [86]:
# trans_revisit(해당 세션이 결제/배송 로그만으로 이루어진 세션인지 여부) 칼럼 생성
# 해당 칼럼 계산 시, 결제/배송 로그만으로 이루어진 세션 중 검색유입 세션(9개 존재)은 제외
raw_all['trans_revisit'] = (raw_all['trans_revisit_sum'] == raw_all['session_length']).astype(float) * (1-raw_all['search_in'])

In [87]:
raw_all.drop_duplicates('id').groupby('trans_revisit')['search_in'].value_counts().unstack()

search_in,0.0,1.0
trans_revisit,,
0.0,228547.0,99357.0
1.0,24003.0,NaN


In [88]:
# 더 이상 쓰지 않을 칼럼들 drop
raw_all.drop(['is_trans_revisit','trans_revisit_sum'],axis=1,inplace=True)

In [89]:
# in_type (검색유입/탐색유입/결제배송 종 어디에 해당하는 세션인지) 칼럼 생성
def compute_in_type(x):
    if x==2:
        return '검색유입'
    elif x==1:
        return '결제/배송'
    elif x==0:
        return '탐색유입'
    else:
        return np.nan

raw_all['in_type'] = (raw_all['search_in']*2 + raw_all['trans_revisit']).apply(lambda x: compute_in_type(x))

In [90]:
pd.DataFrame(raw_all.drop_duplicates('id')['in_type'].value_counts())

,in_type
탐색유입,228547
검색유입,99357
결제/배송,24003


In [91]:
# 더 이상 쓰지 않을 칼럼들 drop
raw_all.drop(['search_in','trans_revisit'],axis=1,inplace=True)

In [92]:
raw_all.shape

(4946055, 67)

In [93]:
raw_all.columns

Index(['datetime', 'user_id', 'sex', 'age', 'url', 'device', 'page',
       'Labeling', 'session', 'id', 'page_stay_time', 'session_length',
       'session_nth', 'prev_Labeling', 'prev_page', 'begin_time', 'end_time',
       'cart_prod_no', 'cart_prod_nm', 'cart_event_type', 'cart_event_ymdt',
       'cart_catg_nm_l', 'cart_catg_nm_m', 'cart_catg_nm_s',
       'log_cart_time_diff', 'like_keep_stat_cd', 'like_prod_no',
       'like_prod_nm', 'like_keep_reg_ymdt', 'like_catg_nm_l',
       'like_catg_nm_m', 'like_catg_nm_s', 'log_like_time_diff',
       'purchase_prod_order_no', 'purchase_order_no', 'purchase_order_ymdt',
       'purchase_prod_no', 'purchase_prod_nm', 'purchase_gmv', 'purchase_flag',
       'purchase_catg_nm_l', 'purchase_catg_nm_m', 'purchase_catg_nm_s',
       'log_purchase_time_diff', 'cart_id', 'like_id', 'purchase_id',
       'distinct_page_num', 'distinct_label_num', 'max_page_stay_time',
       'min_page_stay_time', 'mean_page_stay_time', 'session_stay_time',
    

# 6. csv 파일로 저장 & 메모리 최적화 후 pickle 파일로 저장

In [94]:
# 전체 데이터를 csv 파일 형태로 저장
raw_all.to_csv(os.path.join(os.getcwd(),'final_log(cart buy zzim).csv'),index=False)

In [8]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [96]:
mem_usage(raw_all)

'9760.81 MB'

In [9]:
def int_memory_reduce(data) :
    data_int = data.select_dtypes(include=['int'])
    converted_int = data_int.apply(pd.to_numeric,downcast='unsigned')
    print(f"Before : {mem_usage(data_int)} -> After : {mem_usage(converted_int)}")
    data[converted_int.columns] = converted_int
    return data

## 연속형 데이터 사이즈 축소 함소
def float_memory_reduce(data) :
    data_float = data.select_dtypes(include=['float'])
    converted_float = data_float.apply(pd.to_numeric,downcast='float')
    print(f"Before : {mem_usage(data_float)} -> After : {mem_usage(converted_float)}")
    data[converted_float.columns] = converted_float
    return data

## 문자형 데이터 사이즈 축소 함소
def object_memory_reduce(data) :
    gl_obj = data.select_dtypes(include=['object']).copy()
    converted_obj = pd.DataFrame()
    for col in gl_obj.columns:
        num_unique_values = len(gl_obj[col].unique())
        num_total_values = len(gl_obj[col])
        if num_unique_values / num_total_values < 0.5:
            converted_obj.loc[:,col] = gl_obj[col].astype('category')
        else:
            converted_obj.loc[:,col] = gl_obj[col]
    print(f"Before : {mem_usage(gl_obj)} -> After : {mem_usage(converted_obj)}")
    data[converted_obj.columns] = converted_obj
    return data

In [98]:
raw_all2 = raw_all.copy()
raw_all2 = int_memory_reduce(raw_all2)
raw_all2 = float_memory_reduce(raw_all2)
raw_all2 = object_memory_reduce(raw_all2)

/tmp/ipykernel_5209/1932203267.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  usage_b = pandas_obj.memory_usage(deep=True).sum()
/tmp/ipykernel_5209/1932203267.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  usage_b = pandas_obj.memory_usage(deep=True).sum()


Before : 37.74 MB -> After : 37.74 MB
Before : 943.39 MB -> After : 490.56 MB
Before : 8232.66 MB -> After : 1256.21 MB


In [99]:
mem_usage(raw_all2)

'2392.99 MB'

In [100]:
raw_all2.to_pickle(os.path.join(os.getcwd(),'final_log(cart buy zzim).pkl'))

# 7. 길이 50 이하 세션만 남기기

In [3]:
raw_all = pd.read_csv(os.path.join(os.getcwd(),'final_log(cart buy zzim).csv'))

/tmp/ipykernel_2948/2580486793.py:1: DtypeWarning: Columns (25,27,28,29,30,31,36) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_all = pd.read_csv(os.path.join(os.getcwd(),'final_log(cart buy zzim).csv'))


In [5]:
raw_all_under50 = raw_all[raw_all['session_length']<=50]

In [7]:
raw_all_under50.to_csv(os.path.join(os.getcwd(),'final_log(cart buy zzim)_under50.csv'),index=False)

In [10]:
mem_usage(raw_all_under50)

'7547.21 MB'

In [12]:
raw_all_under50_small = raw_all_under50.copy()
raw_all_under50_small = int_memory_reduce(raw_all_under50_small)
raw_all_under50_small = float_memory_reduce(raw_all_under50_small)
raw_all_under50_small = object_memory_reduce(raw_all_under50_small)

Before : 359.19 MB -> After : 72.53 MB
Before : 690.75 MB -> After : 359.19 MB
Before : 6874.22 MB -> After : 1310.79 MB


In [13]:
mem_usage(raw_all_under50_small)

'1690.70 MB'

In [14]:
raw_all_under50_small.to_pickle(os.path.join(os.getcwd(),'final_log(cart buy zzim)_under50.pkl'))